In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler
import torchvision.models as models
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR, ReduceLROnPlateau,StepLR
#torch.multiprocessing.set_start_method("spawn")
import vgg_mobile
import vgg
from sklearn.model_selection import KFold,StratifiedKFold
import copy

import os
import numpy as np
import pandas as pd
from utils import progress_bar
from skimage import transform as tf

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [2]:
BASE_dir = 'data/processed/'

train = pd.read_json(BASE_dir + 'train.json')
#test = pd.read_json(BASE_dir + 'test.json')

In [3]:
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])

# Size in number of pixels of every isolated object.
def size(arr):     
    return np.sum(arr<-5)
# Feature engineering iso1 and iso2.
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)

# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
train['size'] = train.s1+train.s2

In [3]:
#prepare data
use_cuda= True if torch.cuda.is_available() else False
#use_cuda =False
#dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor 
dtype = torch.FloatTensor 
data=  pd.read_json(BASE_dir + 'train.json')

class iceberg_angle_dataset(Dataset):
    def __init__(self, data,angle,label,size=None, transform=None, test=False): #data: 1604 * 3 *75* 75
        self.data =data
#         self.angle=torch.cat( (torch.from_numpy(angle).type(torch.FloatTensor).unsqueeze(1),torch.from_numpy(size).type(torch.FloatTensor).unsqueeze(1)),1)
        self.angle=torch.from_numpy(angle).type(torch.FloatTensor).unsqueeze(1)
        self.label = torch.from_numpy(label).type(torch.LongTensor)
        self.transform= transform
        self.test= test
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        img, label, angle=  self.data[idx], self.label[idx], self.angle[idx]
        if self.transform is not None:
            #Random Horizontal Flip and Vertical Flip 
            #https://discuss.pytorch.org/t/torch-from-numpy-not-support-negative-strides/3663
            
            #rotate, scale, shear, translation
#             if self.test is False:
#                 angle = np.random.uniform(0,360)
#                 img = tf.rotate(img,angle=angle,resize=False)
#                 scale1 = np.exp(np.random.uniform(np.log(1/1.2), np.log(1.2)))
#                 scale2 = np.exp(np.random.uniform(np.log(1/1.2), np.log(1.1)))
#                 #shear = np.random.uniform(-np.pi/18, np.pi/18)
#                 #tran = np.random.uniform(-5, 5)
#                 #aug = tf.AffineTransform(shear = shear, translation=tran, scale= (scale1, scale2))
#                 aug = tf.AffineTransform(scale= (scale1, scale2))
#                 img = tf.warp(img, inverse_map=aug)
            
#                 if np.random.uniform()>0.5:
#                     img = np.flip(img,axis=1).copy()
#                 if np.random.uniform()>0.5:
#                     img = np.flip(img,axis=2).copy()
            
            if self.test is False:
                if np.random.uniform()>0.5:
                    img = np.flip(img,axis=1).copy()
                if np.random.uniform()>0.5:
                    img = np.flip(img,axis=2).copy()
                rotate = np.random.randint(4, size=1)
                if rotate:
                    img = np.rot90(img,k=rotate,axes=(1,2)).copy()
                pass
        img = torch.from_numpy(img).type(dtype)
#         img = self.transform(img)

        return img, angle,label    
    
    
def stack(row):
    return np.stack(row[['c1','c2','c3']]).reshape(3,75,75)

def raw_to_numpy(data):
    img = []
    data['c1'] = data['band_1'].apply(np.array)
    data['c2'] = data['band_2'].apply(np.array)
    data['c3'] = (data['c1'] + data['c2'])/2
#     data['c3'] = (data['c1'] + data['c2'])/2
    for _, row in data.iterrows():
        img.append(stack(row))
    return np.stack(img)

def transform_compute(img):
    train_mean = img.mean(axis=(0,2,3))
    train_std = img.std(axis=(0,2,3))
    return train_mean, train_std

def data_aug(X, y):    
    X_rot_30 = []
    X_rot_60 = [] 
    X_h = np.flip(X, 3)
    X_v = np.flip(X, 2)
    for i in X:
        X_rot_30.append(tf.rotate(i,angle=90,resize=False))
        X_rot_60.append(tf.rotate(i,angle=270,resize=False))
        
    X_rot_30 = np.stack(X_rot_30)
    X_rot_60 = np.stack(X_rot_60)
    ch_y = np.concatenate((y,y,y,y,y))
    ch_X = np.concatenate((X, X_h, X_v, X_rot_30, X_rot_60))
    return ch_X, ch_y

train_X = raw_to_numpy(data)#.transpose(0,2,3,1)
train_X.shape     #1604 * 3 *75* 75   N*c*H*W
train_y = data['is_iceberg'].values # if iceberg then 1

In [4]:
# train_index=list(range(1300))
# val_index= list(range(1300,1604))
# train_index=list(range(304,1604)) 
# val_index= list(range(304))
# # train_X[train_index].shape

# # data.inc_angle = data.inc_angle.map(lambda x: 0.0 if x == 'na' else x)
# # train_index = np.where(data.inc_angle > 0)[0]
# # val_index = np.where(data.inc_angle <= 0)[0]

# # seed= np.random.RandomState(123)
# # spliter = KFold(n_splits=5,shuffle =True,random_state = seed)
# # train_index, val_index = next(spliter.split(train_X))
# train_mean, train_std = transform_compute(train_X[train_index])
# train_transform = T.Compose([
#     T.Normalize(train_mean, train_std)
# ])

# train_dataset = iceberg_dataset(data= train_X[train_index], label=train_y[train_index], transform=train_transform)
# val_dataset = iceberg_dataset(data= train_X[val_index], label=train_y[val_index], transform=train_transform, test=True)

# train_loader = DataLoader(train_dataset, batch_size = 32, num_workers=3, 
#                           shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)

## add augmentation 
# seed= np.random.RandomState(123)
# spliter = KFold(n_splits=5,shuffle =True,random_state = seed)
# train_index, val_index = next(spliter.split(train_X))

# train_X_af,train_y_af = data_aug(train_X[train_index], train_y[train_index])
# train_mean, train_std = transform_compute(train_X_af)
# train_transform = T.Compose([
#     T.Normalize(train_mean, train_std)
# ])

# train_dataset = iceberg_dataset(data= train_X_af, label=train_y_af, transform=train_transform)
# val_dataset = iceberg_dataset(data= train_X[val_index], label=train_y[val_index], transform=train_transform, test=True)

# train_loader = DataLoader(train_dataset, batch_size = 32, num_workers=3, 
#                           shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)


# train_X_del = train_X[data.inc_angle!='na',:,:,:]
# train_y_del = train_y[data.inc_angle!='na']
train_X_del = train_X
train_y_del = train_y

seed= np.random.RandomState(123)
spliter = KFold(n_splits=5,shuffle =True,random_state = seed)
train_index, val_index = next(spliter.split(train_X_del))
# # train_index=list(range(284,1471)) 
# # val_index= list(range(284))

train_mean, train_std = transform_compute(train_X_del[train_index])
train_transform = T.Compose([
    T.Normalize(train_mean, train_std)
])
# af_train_X, af_train_y = data_aug(train_X_del[train_index], train_y_del[train_index])
#af_train_X, af_train_y = data_aug2(train_X_del[train_index], train_y_del[train_index])
af_train_X, af_train_y = train_X_del[train_index], train_y_del[train_index]

train_dataset = iceberg_dataset(data= af_train_X, label=af_train_y, transform=train_transform)
val_dataset = iceberg_dataset(data= train_X_del[val_index], label=train_y_del[val_index], transform=train_transform, test=True)

train_loader = DataLoader(train_dataset, batch_size = 16, num_workers=3, 
                          shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)

In [28]:
train_X_del = train_X
train_y_del = train_y
train_mean, train_std = transform_compute(train_X_del[train_index])
train_transform = T.Compose([
    T.Normalize(train_mean, train_std)
])

In [11]:
## angle and size

#data['inc_angle'][data['inc_angle']=='na']=0
data.loc[data['inc_angle']=='na', 'inc_angle'] = 0

train_X = train_X
train_angle_del = data['inc_angle'].values
train_angle = train_angle_del.astype(np.float)
#train_size = train['size'].values
train_y = train_y

train_X_del = train_X
train_y_del = train_y

seed= np.random.RandomState(123)
spliter = KFold(n_splits=5,shuffle =True,random_state = seed)
train_index, val_index = next(spliter.split(train_X_del))
# # train_index=list(range(284,1471)) 
# # val_index= list(range(284))

train_mean, train_std = transform_compute(train_X_del[train_index])
train_transform = T.Compose([
    T.Normalize(train_mean, train_std)
])
#af_train_X,af_train_angle, af_train_y = data_aug(train_X_del[train_index], train_angle_del[train_index],train_y_del[train_index])
#af_train_X, af_train_y = data_aug2(train_X_del[train_index], train_y_del[train_index])


train_dataset = iceberg_angle_dataset(data= train_X[train_index], angle=train_angle[train_index],
                                    label=train_y[train_index],
                                    transform=train_transform, test=False)

val_dataset = iceberg_angle_dataset(data= train_X[val_index], angle=train_angle[val_index],
                                    label=train_y[val_index],
                                    transform=train_transform, test=True)

# train_dataset = iceberg_angle_dataset(data= train_X[train_index], angle=train_angle[train_index],size=train_size[train_index],
#                                     label=train_y[train_index],
#                                     transform=train_transform)

# val_dataset = iceberg_angle_dataset(data= train_X[val_index], angle=train_angle[val_index],size= train_size[val_index],
#                                     label=train_y[val_index],
#                                     transform=train_transform, test=True)



train_loader = DataLoader(train_dataset, batch_size = 16, num_workers=3, 
                          shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)


In [ ]:
for i in train_loader:
    print(i)
    break

In [ ]:
torch.cuda.is_available()
torch.from_numpy(train_X).type(torch.FloatTensor)[1].shape
train_X[1]
use_cuda
# for i in train_loader:
#     print(i.size())
#     break

In [6]:
####train with angle and other


def train(epoch,early_stopping = None):
    global train_data#,out,y,predicted
    acc=0
    best_acc =0
    best_val_loss= 100
    loss_hist = []
    val_loss_hist = []
    train_acc_hist = []
    val_acc_hist = []
    train_data={}
    train_data['loss_hist'] = loss_hist
    train_data['val_loss_hist'] = val_loss_hist
    train_data['train_acc_hist'] = train_acc_hist
    train_data['val_acc_hist'] =  val_acc_hist
    e_s= 0
    last_lr = optimizer.param_groups[0]['lr']
    
    for i in range(epoch):
        print('\nThis is epoch:{}'.format(i+1))
        total= 0
        correct=0
        loss_avg= 0
        scheduler.step()
#         scheduler.step(acc)
        if optimizer.param_groups[0]['lr'] < last_lr:
            print('lr change from %f to %f\n' %(last_lr,optimizer.param_groups[0]['lr']))
            last_lr = optimizer.param_groups[0]['lr']
        net.train()
        for j,(batch_x,batch_angle, batch_y) in enumerate(train_loader):
            optimizer.zero_grad()
            batch_angle=batch_angle.type(torch.FloatTensor)
            if use_cuda:
                batch_x,batch_angle, batch_y = batch_x.cuda(),batch_angle.cuda(),batch_y.cuda()
            x = Variable(batch_x)
            angle = Variable(batch_angle)
            y = Variable(batch_y)
            out = net((x, angle))
            loss = criterion(out, y)
            loss_avg += loss.cpu().data[0] *out.size()[0]
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += predicted.eq(y.data).cpu().sum()
            progress_bar(j, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
            if j % 5==0:
                loss_hist.append(loss_avg/total)
            
        train_acc_hist.append(100.*correct/total)
        e_s+=1
        if i %1 == 0:
            acc, val_loss = test(val_loader)
            val_acc_hist.append(acc)
            if acc >best_acc:
                best_acc= acc
                e_s = 0
                print('acc: Save it!')
                torch.save(net.state_dict(), 'vgg_mobile_acc.pth')
            if val_loss <best_val_loss and loss_avg/total <=val_loss :
                best_val_loss= val_loss
                e_s = 0
                print('loss: Save it!')
                torch.save(net.state_dict(), 'vgg_mobile_loss.pth')
            if loss_avg/total >val_loss:
                e_s=0

#             if best_val_loss >= val_loss:
#                 best_val_loss= val_loss
#                 torch.save(net.state_dict(), 'resnet34_loss%d.pth'%i)
        if early_stopping is not None and e_s >= early_stopping:
            return best_val_loss,best_acc,i

    return best_val_loss,best_acc,i
#         if i%50==0 and save:
#             torch.save(net.state_dict(), 'resnet50.pth')
        
def test(val_load):
    net.eval()
    total = 0
    correct = 0
    loss_avg= 0
    for k, (val_x,val_angle, val_y) in enumerate(val_load):
        val_angle=val_angle.type(torch.FloatTensor)
        if use_cuda:
            val_x, val_angle,val_y = val_x.cuda(),val_angle.cuda(), val_y.cuda()
        x = Variable(val_x)
        angle=Variable(val_angle)
        y = Variable(val_y)
        out = net((x,angle))
        if len(out.size())==1:
            out = out.unsqueeze(0)
        loss = criterion(out, y)
        loss_avg += loss.cpu().data[0] *out.size()[0]
        #print(out.size())
        _, predicted = torch.max(out.data, 1)
        correct += predicted.eq(y.data).cpu().sum()
        total += out.size()[0]
        progress_bar(k, len(val_load), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
    train_data['val_loss_hist'].append(loss_avg/total) #also keep track of loss of val set
    acc =  (correct*100.0)/total
    return acc,loss_avg/total

In [ ]:
vgg16 = pre_vgg.vgg16_bn(pretrained=True)
for param in vgg16.parameters():
    print(param.)

In [50]:
# for  i in net.features:
#     print(i)
#     break
# for i in i.parameters():
#     print(i)
len(net.features)

43

In [19]:
#vgg16 = vgg_fcn.vgg16_bn(pretrained=True)
result=[]
for i in range(1):
    vgg16_bn = vgg_mobile.vgg16(pretrained=True)#copy.deepcopy(vgg16)
    vgg16_bn.mobile = vgg_mobile.MobileNet()
    num = 512+1024+1
    vgg16_bn.classifier = nn.Sequential(
                nn.Linear(1537, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(True),
                nn.Dropout(p=0.5),
#                 nn.Linear(1024, 512),
#                 nn.BatchNorm1d(512),
#                 nn.ReLU(True),
#                 nn.Dropout(p=0.5),
#                 nn.Linear(512, 2)
                nn.Linear(1024, 2)
#                 nn.Linear(num, 2)
            )

    net= vgg16_bn
    # net.load_state_dict(torch.load('vgg_fcn_loss.pth'))

    criterion = nn.CrossEntropyLoss()

    # #Adam does not perform so good here   
    # #(0.1, 0.0001) (50, 80, 110, 170) 52 epoch reaches the maximum.
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov= True)
    # optimizer = optim.Adam(net.classifier.parameters(), lr=0.00001, weight_decay=0.0003)
    scheduler = MultiStepLR(optimizer, [10,25,50], gamma=0.1)
#     scheduler = MultiStepLR(optimizer, [20,40,60], gamma=0.1)
    # scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
#     scheduler = ReduceLROnPlateau(optimizer, 'max', patience =10,min_lr= 0.00001)
    #5e-3 86
    if use_cuda:
        criterion.cuda()
        net.cuda()
    #     resnet101 = torch.nn.DataParallel(resnet101, device_ids=range(torch.cuda.device_count()))
    #     cudnn.benchmark = True   

    a = train(epoch=80,early_stopping =20)
    result.append(a)


This is epoch:1
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s2ms|Loss: 0.479 | Acc: 74.279% (953/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.574 | Acc: 72.897% (234/321)
acc: Save it!
loss: Save it!

This is epoch:2
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.359 | Acc: 82.697% (1061/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.788 | Acc: 78.193% (251/321)
acc: Save it!

This is epoch:3
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.336 | Acc: 85.269% (1094/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.818 | Acc: 71.963% (231/321)

This is epoch:4
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.294 | Acc: 87.217% (1119/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 1.807 | Acc: 58.879% (189/321)

This is epoch:5
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.283 | Acc: 87.217% (1119/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.416 | Acc: 82.243% (264/321)
acc: Save it!

[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.171 | Acc: 93.765% (1203/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.248 | Acc: 91.589% (294/321)

This is epoch:43
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.175 | Acc: 93.375% (1198/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.260 | Acc: 90.654% (291/321)

This is epoch:44
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.175 | Acc: 92.206% (1183/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.240 | Acc: 90.654% (291/321)

This is epoch:45
[=========  81/ 81 ======>]Step: 0ms| Tot: 4s1ms|Loss: 0.176 | Acc: 92.673% (1189/1283)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.267 | Acc: 89.408% (287/321)

This is epoch:46


Process Process-1768:
Process Process-1769:
Process Process-1770:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343

In [81]:
result 5* 2* 10*4

[(0.21936874226246297, 91.27725856697819, 39),
 (0.21782404165773006, 91.58878504672897, 55),
 (0.22525541061924254, 90.96573208722741, 53)]

In [83]:
result

[(0.24811193430535147, 90.03115264797508, 29),
 (0.21092650229314405, 91.58878504672897, 28),
 (0.22480700989007207, 90.65420560747664, 53)]

In [69]:
#vgg16 = vgg_fcn.vgg16_bn(pretrained=True)
vgg16_bn = vgg_fcn.vgg16(pretrained=True)#copy.deepcopy(vgg16)

vgg16_bn.classifier = nn.Sequential(
            nn.Linear(512+1, 256),
#             nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Dropout(p=0.3),
            nn.Linear(256, 256),
#             nn.BatchNorm1d(512),
            nn.ReLU(True),
            nn.Dropout(p=0.3),
            nn.Linear(256, 2)
        )

net= vgg16_bn
net.load_state_dict(torch.load('cnn_ang_loss.pth'))
for i in vgg16_bn.features:
    i.requires_grad = False


criterion = nn.CrossEntropyLoss()

# #Adam does not perform so good here   
# #(0.1, 0.0001) (50, 80, 110, 170) 52 epoch reaches the maximum.
optimizer = optim.SGD(net.classifier.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.0003, nesterov= True)
# optimizer = optim.Adam(net.classifier.parameters(), lr=0.00001, weight_decay=0.0003)
scheduler = MultiStepLR(optimizer, [5,10,15], gamma=0.1)
# scheduler = MultiStepLR(optimizer, [8,18], gamma=0.1)
# scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
# scheduler = ReduceLROnPlateau(optimizer, 'max', patience =10,min_lr= 0.0001)
#5e-3 86
if use_cuda:
    criterion.cuda()
    net.cuda()
#     resnet101 = torch.nn.DataParallel(resnet101, device_ids=range(torch.cuda.device_count()))
#     cudnn.benchmark = True   

train(epoch=250,early_stopping =20)


This is epoch:1
[=========  81/ 81 ======>]Step: 0ms| Tot: 3s|Loss: 0.088 | Acc: 97.194% (1247/1283)64)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.218 | Acc: 92.523% (297/321)
acc: Save it!
loss: Save it!

This is epoch:2
[=========  81/ 81 ======>]Step: 0ms| Tot: 3s|Loss: 0.089 | Acc: 97.272% (1248/1283)64)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.222 | Acc: 92.523% (297/321)

This is epoch:3
[=========  81/ 81 ======>]Step: 0ms| Tot: 3s|Loss: 0.089 | Acc: 96.726% (1241/1283)64)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.222 | Acc: 93.146% (299/321)
acc: Save it!

This is epoch:4
[=========  81/ 81 ======>]Step: 0ms| Tot: 3s|Loss: 0.082 | Acc: 97.272% (1248/1283)64)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.225 | Acc: 92.835% (298/321)

This is epoch:5
[=========  81/ 81 ======>]Step: 0ms| Tot: 3s|Loss: 0.088 | Acc: 96.960% (1244/1283)64)
[=========   6/  6 ==>....]Step: 0ms| Tot: 1ms|Loss: 0.226 | Acc: 93.146% (299/321)

This is epo

Process Process-8923:
Process Process-8924:
Process Process-8925:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/proc

KeyboardInterrupt: 

In [75]:
test_set = pd.read_json(BASE_dir + 'test.json')
test_X = raw_to_numpy(test_set)
test_X.shape

(8424, 3, 75, 75)

In [77]:
k =np.stack(result).mean(axis=0)
# #sub.shape
# result[1].shape
# np.concatenate(prob).shape

In [80]:
sub=pd.DataFrame()
sub['id'] = test_set['id']
sub['is_iceberg'] =  np.concatenate(prob)
sub.shape
sub.to_csv('submission2.csv',index=False)

In [41]:
temp= pd.read_csv('submission3.csv') #0.0001 wd one
sub['is_iceberg2'] = temp['is_iceberg']
sub.corr()

,is_iceberg,is_iceberg2
is_iceberg,1.000000,0.886197
is_iceberg2,0.886197,1.000000


In [21]:
def train(epoch,early_stopping = None):
    global train_data#,out,y,predicted
    acc=0
    best_acc =0
    best_val_loss= 100
    loss_hist = []
    val_loss_hist = []
    train_acc_hist = []
    val_acc_hist = []
    train_data={}
    train_data['loss_hist'] = loss_hist
    train_data['val_loss_hist'] = val_loss_hist
    train_data['train_acc_hist'] = train_acc_hist
    train_data['val_acc_hist'] =  val_acc_hist
    e_s= 0
    last_lr = optimizer.param_groups[0]['lr']
    
    for i in range(epoch):
        print('\nThis is epoch:{}'.format(i+1))
        total= 0
        correct=0
        loss_avg= 0
        scheduler.step()
#         scheduler.step(acc)
        if optimizer.param_groups[0]['lr'] < last_lr:
            print('lr change from %f to %f\n' %(last_lr,optimizer.param_groups[0]['lr']))
            last_lr = optimizer.param_groups[0]['lr']
        net.train()
        for j,(batch_x,batch_angle, batch_y) in enumerate(train_loader):
            optimizer.zero_grad()
            batch_angle=batch_angle.type(torch.FloatTensor)
            if use_cuda:
                batch_x,batch_angle, batch_y = batch_x.cuda(),batch_angle.cuda(),batch_y.cuda()
            x = Variable(batch_x)
            angle = Variable(batch_angle)
            y = Variable(batch_y)
            out = net((x, angle))
            loss = criterion(out, y)
            loss_avg += loss.cpu().data[0] *out.size()[0]
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += predicted.eq(y.data).cpu().sum()
            progress_bar(j, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
            if j % 5==0:
                loss_hist.append(loss_avg/total)
            
        train_acc_hist.append(100.*correct/total)
        e_s+=1
        if i %1 == 0:
            acc, val_loss = test(val_loader)
            val_acc_hist.append(acc)
            if acc >best_acc:
                best_acc= acc
                e_s = 0
                print('acc: Save it!')
                torch.save(net.state_dict(), 'vgg_mobile_acc.pth')
            if val_loss <best_val_loss:# and loss_avg/total <=val_loss :
                best_val_loss= val_loss
                e_s = 0
                print('loss: Save it!')
                torch.save(net.state_dict(), 'vgg_mobile_loss.pth')
            if loss_avg/total >val_loss:
                e_s=0

#             if best_val_loss >= val_loss:
#                 best_val_loss= val_loss
#                 torch.save(net.state_dict(), 'resnet34_loss%d.pth'%i)
        if early_stopping is not None and e_s >= early_stopping:
            return best_val_loss,best_acc,i

    return best_val_loss,best_acc,i
#         if i%50==0 and save:
#             torch.save(net.state_dict(), 'resnet50.pth')
        
def test(val_load):
    net.eval()
    total = 0
    correct = 0
    loss_avg= 0
    for k, (val_x,val_angle, val_y) in enumerate(val_load):
        val_angle=val_angle.type(torch.FloatTensor)
        if use_cuda:
            val_x, val_angle,val_y = val_x.cuda(),val_angle.cuda(), val_y.cuda()
        x = Variable(val_x)
        angle=Variable(val_angle)
        y = Variable(val_y)
        out = net((x,angle))
        if len(out.size())==1:
            out = out.unsqueeze(0)
        loss = criterion(out, y)
        loss_avg += loss.cpu().data[0] *out.size()[0]
        #print(out.size())
        _, predicted = torch.max(out.data, 1)
        correct += predicted.eq(y.data).cpu().sum()
        total += out.size()[0]
        progress_bar(k, len(val_load), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
    train_data['val_loss_hist'].append(loss_avg/total) #also keep track of loss of val set
    acc =  (correct*100.0)/total
    return acc,loss_avg/total

#Try different transformation

for rou in range(1):
    ran_num = np.random.randint(120000,130000,size=1)[0]
    seed= np.random.RandomState(ran_num)
    spliter = KFold(n_splits=10,shuffle =True,random_state = seed)
    for k,(train_index, val_index) in enumerate(spliter.split(train_X_del)):
        
        train_dataset = iceberg_angle_dataset(data= train_X[train_index], angle=train_angle[train_index],
                                            label=train_y[train_index],
                                            transform=train_transform, test=False)

        val_dataset = iceberg_angle_dataset(data= train_X[val_index], angle=train_angle[val_index],
                                            label=train_y[val_index],
                                            transform=train_transform, test=True)

        train_loader = DataLoader(train_dataset, batch_size = 16, num_workers=3, 
                                  shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)

        
        candidate = []
        for rep in range(3):
            vgg16_bn = vgg_mobile.vgg16(pretrained=True)#copy.deepcopy(vgg16)
            vgg16_bn.mobile = vgg_mobile.MobileNet()
            num = 512+1024+1
            vgg16_bn.classifier = nn.Sequential(
                        nn.Linear(1537, 1024),
                        nn.BatchNorm1d(1024),
                        nn.ReLU(True),
                        nn.Dropout(p=0.5),
        #                 nn.Linear(1024, 512),
        #                 nn.BatchNorm1d(512),
        #                 nn.ReLU(True),
        #                 nn.Dropout(p=0.5),
        #                 nn.Linear(512, 2)
                        nn.Linear(1024, 2)
        #                 nn.Linear(num, 2)
                    )

            net= vgg16_bn
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov= True)
            scheduler = MultiStepLR(optimizer, [10,25,50], gamma=0.1)
            #5e-3 86
            if use_cuda:
                criterion.cuda()
                net.cuda()
            result = train(epoch=80,early_stopping =20)
            with open("vgg19_models/log.txt", "a") as myfile:
                msg = '10folds, Phase1, At fold {}, seed {},round {} we find one with acc: {}, loss: {}\n'.format(
                                                            k,ran_num,rep+1, result[1], result[0])
                myfile.write(msg)
            cmd = 'cp vgg_mobile_loss.pth vgg_mobile_loss{}.pth'.format(rep)
            os.system(cmd)
            del vgg16_bn
        
        for g in range(3):
            cmd = 'cp vgg_mobile_loss{}.pth vgg_mobile_models/r1_10vgg_mob{}_{}{}.pth'.format(g,rou,k,g)
            os.system(cmd)
            


This is epoch:1
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.482 | Acc: 73.389% (1059/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 1.459 | Acc: 57.764% (93/161)
acc: Save it!
loss: Save it!

This is epoch:2
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.370 | Acc: 83.576% (1206/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.857 | Acc: 73.292% (118/161)
acc: Save it!
loss: Save it!

This is epoch:3
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.368 | Acc: 84.130% (1214/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.442 | Acc: 83.851% (135/161)
acc: Save it!
loss: Save it!

This is epoch:4
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.298 | Acc: 86.625% (1250/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.492 | Acc: 81.988% (132/161)

This is epoch:5
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.270 | Acc: 88.565% (1278/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 

[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.160 | Acc: 91.925% (148/161)

This is epoch:43
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.176 | Acc: 93.902% (1355/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.164 | Acc: 91.925% (148/161)

This is epoch:44
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.177 | Acc: 93.416% (1348/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.174 | Acc: 91.925% (148/161)

This is epoch:45
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.177 | Acc: 93.139% (1344/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.177 | Acc: 93.168% (150/161)

This is epoch:46
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.177 | Acc: 93.001% (1342/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.159 | Acc: 93.789% (151/161)

This is epoch:47
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.174 | Acc: 93.001% (1342/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Los

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.286 | Acc: 87.249% (1259/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.261 | Acc: 87.578% (141/161)
acc: Save it!
loss: Save it!

This is epoch:6
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.284 | Acc: 87.803% (1267/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.231 | Acc: 91.304% (147/161)
acc: Save it!
loss: Save it!

This is epoch:7
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.300 | Acc: 87.872% (1268/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.195 | Acc: 91.304% (147/161)
loss: Save it!

This is epoch:8
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.261 | Acc: 88.912% (1283/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.237 | Acc: 90.683% (146/161)

This is epoch:9
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.255 | Acc: 88.565% (1278/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.380 | Acc: 81.366% (131/161)

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.189 | Acc: 92.516% (1335/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.158 | Acc: 92.547% (149/161)

This is epoch:47
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.184 | Acc: 92.169% (1330/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.153 | Acc: 94.410% (152/161)

This is epoch:48
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.184 | Acc: 93.070% (1343/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.152 | Acc: 94.410% (152/161)

This is epoch:49
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.190 | Acc: 92.793% (1339/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.156 | Acc: 93.168% (150/161)

This is epoch:50
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.188 | Acc: 92.377% (1333/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.152 | Acc: 93.168% (150/161)

This is epoch:51
lr change from 0.000010 to 0.000001

[=========  9

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.244 | Acc: 89.605% (1293/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.340 | Acc: 84.472% (136/161)

This is epoch:9
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.248 | Acc: 89.258% (1288/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.189 | Acc: 91.925% (148/161)
acc: Save it!
loss: Save it!

This is epoch:10
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.242 | Acc: 89.951% (1298/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.242 | Acc: 86.957% (140/161)

This is epoch:11
lr change from 0.001000 to 0.000100

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.235 | Acc: 90.644% (1308/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.212 | Acc: 90.062% (145/161)

This is epoch:12
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.229 | Acc: 91.199% (1316/1443)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.227 | Acc: 88.820% (143/161)

Th

...

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.163 | Acc: 93.906% (1356/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.195 | Acc: 93.125% (149/160)

This is epoch:41
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.171 | Acc: 93.837% (1355/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.195 | Acc: 93.125% (149/160)

This is epoch:42
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.165 | Acc: 93.906% (1356/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.194 | Acc: 93.125% (149/160)

This is epoch:43
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.179 | Acc: 93.075% (1344/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.193 | Acc: 92.500% (148/160)

This is epoch:44
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.172 | Acc: 93.767% (1354/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.193 | Acc: 93.125% (149/160)

This is epoch:45
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|L

[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.292 | Acc: 90.000% (144/160)
acc: Save it!

This is epoch:23
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.201 | Acc: 92.105% (1330/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.292 | Acc: 88.750% (142/160)

This is epoch:24
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.191 | Acc: 93.283% (1347/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.264 | Acc: 86.875% (139/160)

This is epoch:25
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.184 | Acc: 92.105% (1330/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.241 | Acc: 90.000% (144/160)
loss: Save it!

This is epoch:26
lr change from 0.000100 to 0.000010

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.202 | Acc: 92.521% (1336/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.271 | Acc: 89.375% (143/160)

This is epoch:27
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.182 | Acc: 92.7

[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.278 | Acc: 88.750% (142/160)

This is epoch:65
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.177 | Acc: 93.075% (1344/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.270 | Acc: 89.375% (143/160)

This is epoch:66
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.183 | Acc: 92.798% (1340/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.225 | Acc: 90.000% (144/160)

This is epoch:67
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.172 | Acc: 93.144% (1345/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.224 | Acc: 90.000% (144/160)

This is epoch:68
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.177 | Acc: 93.006% (1343/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.252 | Acc: 88.750% (142/160)

This is epoch:1
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.494 | Acc: 72.576% (1048/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.190 | Acc: 92.729% (1339/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.184 | Acc: 89.375% (143/160)

This is epoch:39
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.182 | Acc: 92.798% (1340/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.191 | Acc: 90.625% (145/160)

This is epoch:40
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.169 | Acc: 93.837% (1355/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.186 | Acc: 89.375% (143/160)

This is epoch:41
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.162 | Acc: 93.906% (1356/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.186 | Acc: 90.000% (144/160)

This is epoch:42
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s6ms|Loss: 0.171 | Acc: 93.698% (1353/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.191 | Acc: 90.625% (145/160)

This is epoch:43
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|L

[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.198 | Acc: 91.551% (1322/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.238 | Acc: 89.375% (143/160)

This is epoch:22
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.186 | Acc: 93.698% (1353/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.221 | Acc: 90.000% (144/160)
loss: Save it!

This is epoch:23
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.187 | Acc: 92.729% (1339/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.231 | Acc: 88.750% (142/160)

This is epoch:24
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.195 | Acc: 92.175% (1331/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.208 | Acc: 90.625% (145/160)
loss: Save it!

This is epoch:25
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s7ms|Loss: 0.181 | Acc: 93.560% (1351/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.231 | Acc: 89.375% (143/160)

This is epoch:26
lr change from 0.000

In [10]:
def train(epoch,early_stopping = None):
    global train_data#,out,y,predicted
    acc=0
    best_acc =0
    best_val_loss= 100
    loss_hist = []
    val_loss_hist = []
    train_acc_hist = []
    val_acc_hist = []
    train_data={}
    train_data['loss_hist'] = loss_hist
    train_data['val_loss_hist'] = val_loss_hist
    train_data['train_acc_hist'] = train_acc_hist
    train_data['val_acc_hist'] =  val_acc_hist
    e_s= 0
    last_lr = optimizer.param_groups[0]['lr']
    
    for i in range(epoch):
        print('\nThis is epoch:{}'.format(i+1))
        total= 0
        correct=0
        loss_avg= 0
        scheduler.step()
#         scheduler.step(acc)
        if optimizer.param_groups[0]['lr'] < last_lr:
            print('lr change from %f to %f\n' %(last_lr,optimizer.param_groups[0]['lr']))
            last_lr = optimizer.param_groups[0]['lr']
        net.train()
        for j,(batch_x,batch_angle, batch_y) in enumerate(train_loader):
            optimizer.zero_grad()
            batch_angle=batch_angle.type(torch.FloatTensor)
            if use_cuda:
                batch_x,batch_angle, batch_y = batch_x.cuda(),batch_angle.cuda(),batch_y.cuda()
            x = Variable(batch_x)
            angle = Variable(batch_angle)
            y = Variable(batch_y)
            out = net((x, angle))
            loss = criterion(out, y)
            loss_avg += loss.cpu().data[0] *out.size()[0]
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += predicted.eq(y.data).cpu().sum()
            progress_bar(j, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
            if j % 5==0:
                loss_hist.append(loss_avg/total)
            
        train_acc_hist.append(100.*correct/total)
        e_s+=1
        if i %1 == 0:
            acc, val_loss = test(val_loader)
            val_acc_hist.append(acc)
            if acc >best_acc:
                best_acc= acc
                e_s = 0
                print('acc: Save it!')
                torch.save(net.state_dict(), 'vgg19_acc.pth')
            if val_loss <best_val_loss:# and loss_avg/total <=val_loss :
                best_val_loss= val_loss
                e_s = 0
                print('loss: Save it!')
                torch.save(net.state_dict(), 'vgg19_loss.pth')
            if loss_avg/total >val_loss:
                e_s=0

#             if best_val_loss >= val_loss:
#                 best_val_loss= val_loss
#                 torch.save(net.state_dict(), 'resnet34_loss%d.pth'%i)
        if early_stopping is not None and e_s >= early_stopping:
            return best_val_loss,best_acc,i

    return best_val_loss,best_acc,i
#         if i%50==0 and save:
#             torch.save(net.state_dict(), 'resnet50.pth')
        
def test(val_load):
    net.eval()
    total = 0
    correct = 0
    loss_avg= 0
    for k, (val_x,val_angle, val_y) in enumerate(val_load):
        val_angle=val_angle.type(torch.FloatTensor)
        if use_cuda:
            val_x, val_angle,val_y = val_x.cuda(),val_angle.cuda(), val_y.cuda()
        x = Variable(val_x)
        angle=Variable(val_angle)
        y = Variable(val_y)
        out = net((x,angle))
        if len(out.size())==1:
            out = out.unsqueeze(0)
        loss = criterion(out, y)
        loss_avg += loss.cpu().data[0] *out.size()[0]
        #print(out.size())
        _, predicted = torch.max(out.data, 1)
        correct += predicted.eq(y.data).cpu().sum()
        total += out.size()[0]
        progress_bar(k, len(val_load), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (loss_avg/total, 100.*correct/total, correct, total))
    train_data['val_loss_hist'].append(loss_avg/total) #also keep track of loss of val set
    acc =  (correct*100.0)/total
    return acc,loss_avg/total

#Try different transformation

for rou in range(1):
    if rou ==0:
        ran_num = 59790
    else:
        ran_num = np.random.randint(50000,60000, size=1)[0]
    seed= np.random.RandomState(ran_num)
    spliter = KFold(n_splits=10,shuffle =True,random_state = seed)
    for k,(train_index, val_index) in enumerate(spliter.split(train_X_del)):
        if rou ==0 and k not in [4]:
            print('pass %d:%d' %(rou,k))
            continue
        train_dataset = iceberg_angle_dataset(data= train_X[train_index], angle=train_angle[train_index],
                                            label=train_y[train_index],
                                            transform=train_transform, test=True)

        val_dataset = iceberg_angle_dataset(data= train_X[val_index], angle=train_angle[val_index],
                                            label=train_y[val_index],
                                            transform=train_transform, test=True)

        train_loader = DataLoader(train_dataset, batch_size = 16, num_workers=3, 
                                  shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size = 64, num_workers=3)

        
        candidate = []
        for rep in range(3):
            vgg16_bn = vgg_fcn.vgg19(pretrained=True)#copy.deepcopy(vgg16)
            num = 256
            vgg16_bn.classifier = nn.Sequential(
                        nn.Linear(512+1, num),
                        nn.BatchNorm1d(num),
                        nn.ReLU(True),
                        nn.Dropout(p=0.3),
                        nn.Linear(num, num),
                        nn.BatchNorm1d(num),
                        nn.ReLU(True),
                        nn.Dropout(p=0.5),
                        nn.Linear(num, 2)
                    )
            net= vgg16_bn
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.00001, nesterov= True)
            scheduler = MultiStepLR(optimizer, [5,11,20], gamma=0.1)
            #5e-3 86
            if use_cuda:
                criterion.cuda()
                net.cuda()
            result = train(epoch=75,early_stopping =20)
            candidate.append(result[0])
            with open("vgg19_models/log.txt", "a") as myfile:
                msg = '10folds, Phase3, At fold {}, seed {},round {} we find one with acc: {}, loss: {}\n'.format(
                                                            k,ran_num,rep+1, result[1], result[0])
                myfile.write(msg)
            cmd = 'cp vgg19_loss.pth vgg19_loss{}.pth'.format(rep)
            os.system(cmd)
            del vgg16_bn
            if rep ==1 and np.sum(np.array(candidate)>0.20)>=1:
                continue
            elif rep==1:
                break
            
        for g in range(rep+1):
            cmd = 'cp vgg19_loss{}.pth vgg19_models/r3_10vgg{}_{}{}.pth'.format(g,rou,k,g)
            os.system(cmd)
            

pass 0:0
pass 0:1
pass 0:2
pass 0:3

This is epoch:1
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s5ms|Loss: 0.512 | Acc: 73.615% (1063/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 1.347 | Acc: 58.125% (93/160)
acc: Save it!
loss: Save it!

This is epoch:2
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s5ms|Loss: 0.366 | Acc: 85.457% (1234/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.417 | Acc: 81.875% (131/160)
acc: Save it!
loss: Save it!

This is epoch:3
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s5ms|Loss: 0.293 | Acc: 88.227% (1274/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.340 | Acc: 86.250% (138/160)
acc: Save it!
loss: Save it!

This is epoch:4
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s5ms|Loss: 0.241 | Acc: 90.305% (1304/1444)
[=========   3/  3 .......]Step: 0ms| Tot: 0ms|Loss: 0.364 | Acc: 85.625% (137/160)

This is epoch:5
[=========  91/ 91 ======>]Step: 0ms| Tot: 4s5ms|Loss: 0.265 | Acc: 88.989% (1285/1444)
[=========   3/  

In [28]:
temp11 = pd.DataFrame()
# temp11= pd.read_csv('plain_cnn_15_models.csv')
test = pd.read_json(BASE_dir + 'test.json')
test_X = raw_to_numpy(test)
test_X.shape 
fake_label = np.zeros(len(test_X))

test_dataset = iceberg_dataset(data= test_X, label=fake_label, transform=train_transform,test=True)

test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=3)




# waiting_list=  


#waiting_list = [i for i in os.listdir('vgg_models/') if 'r1' in i]
 
vgg16_bn = vgg_mobile.vgg16(pretrained=True)#copy.deepcopy(vgg16)
vgg16_bn.mobile = vgg_mobile.MobileNet()
num = 512+1024+1
vgg16_bn.classifier = nn.Sequential(
            nn.Linear(1537, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
#                 nn.Linear(1024, 512),
#                 nn.BatchNorm1d(512),
#                 nn.ReLU(True),
#                 nn.Dropout(p=0.5),
#                 nn.Linear(512, 2)
            nn.Linear(1024, 2)
#                 nn.Linear(num, 2)
        )

# vgg16_bn.avg= nn.Conv2d(512, 512, kernel_size=2,
#                                bias=False)

# vgg16_bn.classifier = nn.Sequential(
#             nn.Linear(512, 512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(True),
#             nn.Dropout(p=0.5),
#             nn.Linear(512, 512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(True),
#             nn.Dropout(p=0.6),
#             nn.Linear(512, 2)
#         )

waiting_list=['r1_10vgg_mob0_00.pth','r1_10vgg_mob0_50.pth',
              'r1_10vgg_mob0_12.pth','r1_10vgg_mob0_62.pth',
              'r1_10vgg_mob0_22.pth','r1_10vgg_mob0_72.pth',
              'r1_10vgg_mob0_32.pth','r1_10vgg_mob0_81.pth',
              'r1_10vgg_mob0_42.pth','r1_10vgg_mob0_91.pth',
             ]
waiting_list= [os.path.join('vgg_mobile_models', i) for i in waiting_list]


net= vgg16_bn

print(waiting_list)
for i,pth in enumerate(waiting_list):
    net.load_state_dict(torch.load(pth))
    net.cuda()
    prob = [] 
    net.eval()
    for k, (val_x, val_y) in enumerate(test_loader):
        if use_cuda:
            val_x, val_y = val_x.cuda(), val_y.cuda()
        x = Variable(val_x)
        y = Variable(val_y)
        out = net(x)
        #prevent overflow
        temp = np.exp(out.cpu().data.numpy()-np.max(out.cpu().data.numpy(),axis=1)[:,np.newaxis])
        ans= temp[:,1]/(temp.sum(axis=1))
        prob.append(ans)
        #print(out.size())
        progress_bar(k, len(test_loader))
    msg = 'is_iceberg%d' % (i)
    temp11[msg]= np.concatenate(prob)

['vgg_models/r1_5vgg0_00.pth', 'vgg_models/r1_5vgg0_11.pth', 'vgg_models/r1_5vgg0_20.pth', 'vgg_models/r1_5vgg0_31.pth', 'vgg_models/r1_5vgg0_40.pth']
[========= 132/132 ======>]Step: 0ms| Tot: 5s6mss
[========= 132/132 ======>]Step: 0ms| Tot: 5s6ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s6ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s7ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s7ms


In [25]:
temp11 = pd.DataFrame()
# temp11= pd.read_csv('plain_cnn_15_models.csv')
test = pd.read_json(BASE_dir + 'test.json')
test_X = raw_to_numpy(test)
test_X.shape 
fake_label = np.zeros(len(test_X))

test_dataset = iceberg_angle_dataset(data= test_X, label=fake_label,angle=test.inc_angle.values.astype(np.float), transform=train_transform,test=True)

# data = test
# data.loc[data.inc_angle=='na', 'inc_angle']=0
# train_angle_del = data['inc_angle'].values
# train_angle_del = train_angle_del.astype(np.float)
# test_dataset = iceberg_angle_dataset(data= test_X, label=fake_label,angle= train_angle_del, transform=train_transform,test=True)




test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=3)


vgg16_bn = vgg_mobile.vgg16(pretrained=True)#copy.deepcopy(vgg16)
vgg16_bn.mobile = vgg_mobile.MobileNet()
num = 512+1024+1
vgg16_bn.classifier = nn.Sequential(
            nn.Linear(1537, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
#                 nn.Linear(1024, 512),
#                 nn.BatchNorm1d(512),
#                 nn.ReLU(True),
#                 nn.Dropout(p=0.5),
#                 nn.Linear(512, 2)
            nn.Linear(1024, 2)
#                 nn.Linear(num, 2)
        )

# vgg16_bn.avg= nn.Conv2d(512, 512, kernel_size=2,
#                                bias=False)

# vgg16_bn.classifier = nn.Sequential(
#             nn.Linear(512, 512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(True),
#             nn.Dropout(p=0.5),
#             nn.Linear(512, 512),
#             nn.BatchNorm1d(512),
#             nn.ReLU(True),
#             nn.Dropout(p=0.6),
#             nn.Linear(512, 2)
#         )

waiting_list=['r1_10vgg_mob0_00.pth','r1_10vgg_mob0_50.pth',
              'r1_10vgg_mob0_12.pth','r1_10vgg_mob0_62.pth',
              'r1_10vgg_mob0_22.pth','r1_10vgg_mob0_72.pth',
              'r1_10vgg_mob0_32.pth','r1_10vgg_mob0_81.pth',
              'r1_10vgg_mob0_42.pth','r1_10vgg_mob0_91.pth',
             ]
waiting_list= [os.path.join('vgg_mobile_models', i) for i in waiting_list]


# waiting_list += ['r3_10vgg0_00.pth','r3_10vgg0_50.pth',
#                 'r3_10vgg0_11.pth','r3_10vgg0_60.pth',
#                 'r3_10vgg0_20.pth','r3_10vgg0_71.pth',
#                 'r3_10vgg0_31.pth','r3_10vgg0_80.pth',
#                 'r3_10vgg0_42.pth','r3_10vgg0_91.pth',
#                 'r3_10vgg1_00.pth','r3_10vgg1_51.pth',
#                 'r3_10vgg1_11.pth','r3_10vgg1_60.pth',
#                 'r3_10vgg1_20.pth','r3_10vgg1_70.pth',
#                 'r3_10vgg1_31.pth','r3_10vgg1_81.pth',
#                 'r3_10vgg1_40.pth','r3_10vgg1_91.pth',
#                 'r3_10vgg2_01.pth','r3_10vgg2_52.pth',
#                 'r3_10vgg2_11.pth','r3_10vgg2_60.pth',
#                 'r3_10vgg2_21.pth','r3_10vgg2_71.pth',
#                 'r3_10vgg2_30.pth','r3_10vgg2_80.pth',
#                 'r3_10vgg2_40.pth','r3_10vgg2_91.pth',
#                 'r3_10vgg3_01.pth','r3_10vgg3_51.pth',
#                 'r3_10vgg3_12.pth','r3_10vgg3_60.pth',
#                 'r3_10vgg3_20.pth','r3_10vgg3_70.pth',
#                 'r3_10vgg3_30.pth','r3_10vgg3_80.pth',
#                 'r3_10vgg3_40.pth','r3_10vgg3_91.pth'
# ]

#waiting_list = [i for i in os.listdir('vgg_models') if 'r3' in i]
#waiting_list = [i for i in os.listdir('vgg_models/') if 'r1' in i]

net= vgg16_bn

for i,pth in enumerate(waiting_list):
    net.load_state_dict(torch.load(pth))
    net.cuda()
    prob = [] 
    net.eval()
    for k, (val_x,val_angle, val_y) in enumerate(test_loader):
        val_angle=val_angle.type(torch.FloatTensor)
        if use_cuda:
            val_x, val_angle,val_y = val_x.cuda(),val_angle.cuda(), val_y.cuda()
        x = Variable(val_x)
        angle=Variable(val_angle)
        y = Variable(val_y)
        out = net((x,angle))
        #prevent overflow
        temp = np.exp(out.cpu().data.numpy()-np.max(out.cpu().data.numpy(),axis=1)[:,np.newaxis])
        ans= temp[:,1]/(temp.sum(axis=1))
        prob.append(ans)
        #print(out.size())
        progress_bar(k, len(test_loader))
    msg = 'is_iceberg%d' % (i)
    temp11[msg]= np.concatenate(prob)

[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms
[========= 132/132 ======>]Step: 0ms| Tot: 5s9ms


In [26]:
temp11.corr()
# [i for i in os.listdir('vgg_models') if 'r3' in i]

,is_iceberg0,is_iceberg1,is_iceberg2,is_iceberg3,is_iceberg4,is_iceberg5,is_iceberg6,is_iceberg7,is_iceberg8,is_iceberg9
is_iceberg0,1.000000,0.874303,0.828842,0.799959,0.766572,0.793589,0.822450,0.868137,0.807195,0.792869
is_iceberg1,0.874303,1.000000,0.844789,0.847918,0.812201,0.868104,0.869239,0.899507,0.824763,0.781628
is_iceberg2,0.828842,0.844789,1.000000,0.811337,0.890611,0.840400,0.863374,0.925623,0.839119,0.926397
is_iceberg3,0.799959,0.847918,0.811337,1.000000,0.794990,0.844487,0.854538,0.860040,0.819201,0.765688
is_iceberg4,0.766572,0.812201,0.890611,0.794990,1.000000,0.865634,0.880776,0.891302,0.802504,0.890876
is_iceberg5,0.793589,0.868104,0.840400,0.844487,0.865634,1.000000,0.939589,0.890250,0.806371,0.797363
is_iceberg6,0.822450,0.869239,0.863374,0.854538,0.880776,0.939589,1.000000,0.919723,0.835840,0.836885
is_iceberg7,0.868137,0.899507,0.925623,0.860040,0.891302,0.890250,0.919723,1.000000,0.889653,0.897323
is_iceberg8,0.807195,0.824763,0.839119,0.819201,0.802504,0.806371,0.835840,0.889653,1.000000,0.829701
is_iceberg9,0.792869,0.781628,0.926397,0.765688,0.890876,0.797363,0.836885,0.897323,0.829701,1.000000


In [7]:
#result_hist

temp11 = pd.DataFrame()

for i in range(5):
    net = resnet.resnet34(num_classes=2)
    net.load_state_dict(torch.load('resnet34_acc%d.pth'%i))
    net.cuda()

    test = pd.read_json(BASE_dir + 'test.json')
    test_X = raw_to_numpy(test)
    test_X.shape 
    fake_label = np.zeros(len(test_X))

    test_dataset = iceberg_dataset(data= test_X, label=fake_label, transform=train_transform,test=True)

    test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=3)

    prob = [] 
    net.eval()
    for k, (val_x, val_y) in enumerate(test_loader):
        if use_cuda:
            val_x, val_y = val_x.cuda(), val_y.cuda()
        x = Variable(val_x)
        y = Variable(val_y)
        out = net(x)
        #prevent overflow
        temp = np.exp(out.cpu().data.numpy()-np.max(out.cpu().data.numpy(),axis=1)[:,np.newaxis])
        ans= temp[:,1]/(temp.sum(axis=1))
        prob.append(ans)
        #print(out.size())
        progress_bar(k, len(test_loader))
    msg = 'is_iceberg%d' %i
    temp11[msg]= np.concatenate(prob)

[=================== 132/132 ================>]  Step: 162ms | Tot: 27s494ms
[=================== 132/132 ================>]  Step: 160ms | Tot: 27s661ms
[=================== 132/132 ================>]  Step: 162ms | Tot: 27s644ms
[=================== 132/132 ================>]  Step: 162ms | Tot: 27s598ms
[=================== 132/132 ================>]  Step: 161ms | Tot: 27s668ms


In [16]:
sub=pd.DataFrame()
sub['id'] = test['id']
sub['is_iceberg'] = temp11.median(axis=1)
sub.shape
sub.to_csv('submission23.csv',index=False)

In [25]:
temp11['is_iceberg_max'] = temp11.iloc[:, 0:6].max(axis=1)
temp11['is_iceberg_min'] = temp11.iloc[:, 0:6].min(axis=1)
temp11['is_iceberg_median'] = temp11.iloc[:, 0:6].median(axis=1)
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

temp11['is_iceberg_base'] = temp11['is_iceberg5']
temp11['is_iceberg'] = np.where(np.all(temp11.iloc[:,0:6] > cutoff_lo, axis=1), 
                                    temp11['is_iceberg_max'], 
                                    np.where(np.all(temp11.iloc[:,0:6] < cutoff_hi, axis=1),
                                             temp11['is_iceberg_min'], 
                                             temp11['is_iceberg_base']))


In [28]:
sub=pd.DataFrame()
sub['id'] = test['id']
sub['is_iceberg'] = temp11['is_iceberg5']
sub.shape
sub.to_csv('submission5.csv',index=False)

In [8]:
net = resnet.resnet34(num_classes=2)
net.load_state_dict(torch.load('save_resnet34_acc117.pth'))
net.cuda()

test = pd.read_json(BASE_dir + 'test.json')
test_X = raw_to_numpy(test)
test_X.shape 
fake_label = np.zeros(len(test_X))

test_dataset = iceberg_dataset(data= test_X, label=fake_label, transform=train_transform,test=True)

test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=3)

prob = [] 
net.eval()
for k, (val_x, val_y) in enumerate(test_loader):
    if use_cuda:
        val_x, val_y = val_x.cuda(), val_y.cuda()
    x = Variable(val_x)
    y = Variable(val_y)
    out = net(x)
    #prevent overflow
    temp = np.exp(out.cpu().data.numpy()-np.max(out.cpu().data.numpy(),axis=1)[:,np.newaxis])
    ans= temp[:,1]/(temp.sum(axis=1))
    prob.append(ans)
    #print(out.size())
    progress_bar(k, len(test_loader))
msg = 'is_iceberg%d' %5
temp11[msg]= np.concatenate(prob)

[=================== 132/132 ================>]  Step: 162ms | Tot: 27s704ms


In [ ]:
temp11.iloc[:,0:5]

In [27]:
temp11.corr()

,is_iceberg0,is_iceberg1,is_iceberg2,is_iceberg3,is_iceberg4,is_iceberg5,is_iceberg_max,is_iceberg_min,is_iceberg_median,is_iceberg_base,is_iceberg
is_iceberg0,1.000000,0.852644,0.822586,0.648968,0.883101,0.905277,0.682861,0.922862,0.942663,0.905277,0.905900
is_iceberg1,0.852644,1.000000,0.905401,0.754710,0.833295,0.815734,0.821258,0.777728,0.956190,0.815734,0.816630
is_iceberg2,0.822586,0.905401,1.000000,0.771766,0.774018,0.784324,0.847868,0.738630,0.918857,0.784324,0.785453
is_iceberg3,0.648968,0.754710,0.771766,1.000000,0.685649,0.556919,0.940914,0.592617,0.749656,0.556919,0.559032
is_iceberg4,0.883101,0.833295,0.774018,0.685649,1.000000,0.826391,0.685683,0.920097,0.909537,0.826391,0.827514
is_iceberg5,0.905277,0.815734,0.784324,0.556919,0.826391,1.000000,0.653849,0.895245,0.896220,1.000000,0.999683
is_iceberg_max,0.682861,0.821258,0.847868,0.940914,0.685683,0.653849,1.000000,0.583326,0.792055,0.653849,0.655435
is_iceberg_min,0.922862,0.777728,0.738630,0.592617,0.920097,0.895245,0.583326,1.000000,0.875356,0.895245,0.895989
is_iceberg_median,0.942663,0.956190,0.918857,0.749656,0.909537,0.896220,0.792055,0.875356,1.000000,0.896220,0.897011
is_iceberg_base,0.905277,0.815734,0.784324,0.556919,0.826391,1.000000,0.653849,0.895245,0.896220,1.000000,0.999683


In [11]:
seed= np.random.RandomState(67)
spliter = KFold(n_splits=5,shuffle =True,random_state = seed)
for i in spliter.split(list(range(100))):
    print(i)
    break

(array([ 1,  2,  5,  6,  7,  8,  9, 10, 12, 13, 15, 16, 18, 19, 20, 21, 22,
       23, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42,
       44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62,
       63, 65, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 98, 99]), array([ 0,  3,  4, 11, 14, 17, 24, 29, 40, 43, 48, 59, 64, 66, 70, 79, 82,
       83, 93, 97]))


In [19]:
temp11 = pd.DataFrame()

test = pd.read_json(BASE_dir + 'test.json')
test_X = raw_to_numpy(test)
test_X.shape 
fake_label = np.zeros(len(test_X))

test_dataset = iceberg_dataset(data= test_X, label=fake_label, transform=train_transform,test=True)

test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=3)


for i,pth in enumerate([os.path.join('resnet34_save_model',i) for i in os.listdir(path='resnet34_save_model') if '.pth' in i]):
    net = resnet.resnet34(num_classes=2)
    net.load_state_dict(torch.load(pth))
    net.cuda()
    prob = [] 
    net.eval()
    for k, (val_x, val_y) in enumerate(test_loader):
        if use_cuda:
            val_x, val_y = val_x.cuda(), val_y.cuda()
        x = Variable(val_x)
        y = Variable(val_y)
        out = net(x)
        #prevent overflow
        temp = np.exp(out.cpu().data.numpy()-np.max(out.cpu().data.numpy(),axis=1)[:,np.newaxis])
        ans= temp[:,1]/(temp.sum(axis=1))
        prob.append(ans)
        #print(out.size())
        progress_bar(k, len(test_loader))
    msg = 'is_iceberg%d' % i
    temp11[msg]= np.concatenate(prob)

[=================== 132/132 ================>]  Step: 155ms | Tot: 26s222ms Step: 200ms | Tot: 2s492ms  Step: 199ms | Tot: 2s692ms  Step: 200ms | Tot: 7s692ms  Step: 199ms | Tot: 13s642ms  Step: 200ms | Tot: 14s841ms  Step: 200ms | Tot: 24s62ms
[=================== 132/132 ================>]  Step: 152ms | Tot: 26s199ms Step: 199ms | Tot: 1s425ms  Step: 200ms | Tot: 1s626ms  Step: 200ms | Tot: 8s236ms  Step: 200ms | Tot: 12s639ms  Step: 200ms | Tot: 14s844ms  Step: 199ms | Tot: 16s449ms  Step: 199ms | Tot: 17s647ms  Step: 199ms | Tot: 20s857ms  Step: 200ms | Tot: 23s61ms  Step: 199ms | Tot: 25s648ms  Step: 201ms | Tot: 25s850ms
[=================== 132/132 ================>]  Step: 151ms | Tot: 26s141ms Step: 199ms | Tot: 6s583ms  Step: 199ms | Tot: 6s982ms 41/132   Step: 200ms | Tot: 9s576ms  Step: 200ms | Tot: 11s185ms
[=================== 132/132 ================>]  Step: 155ms | Tot: 26s219ms Step: 200ms | Tot: 3s394ms  Step: 200ms | Tot: 4s393ms 52/132   Step: 200ms | Tot: 10s614

In [26]:
sub=pd.DataFrame()
sub['id'] = test['id']
sub['is_iceberg'] = temp11['is_iceberg']
sub.shape
sub.to_csv('submission2.csv',index=False)

In [24]:
result = temp11.mean(1)
temp11.head()

,is_iceberg0,is_iceberg1,is_iceberg2,is_iceberg3
0,7.027504e-03,9.244031e-02,1.784263e-02,5.578169e-03
1,3.931345e-03,3.659658e-01,2.564293e-01,1.571568e-02
2,5.239599e-10,1.970750e-21,3.803356e-08,2.089403e-21
3,9.993261e-01,9.456407e-01,9.853242e-01,9.989353e-01
4,1.448082e-03,6.435396e-02,3.096765e-02,2.362306e-04


In [25]:
temp11['is_iceberg_max'] = temp11.iloc[:, :4].max(axis=1)
temp11['is_iceberg_min'] = temp11.iloc[:, :4].min(axis=1)
temp11['is_iceberg_median'] = temp11.iloc[:, :4].median(axis=1)
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

temp11['is_iceberg_base'] = temp11['is_iceberg3']
temp11['is_iceberg'] = np.where(np.all(temp11.iloc[:,0:6] > cutoff_lo, axis=1), 
                                    temp11['is_iceberg_max'], 
                                    np.where(np.all(temp11.iloc[:,0:6] < cutoff_hi, axis=1),
                                             temp11['is_iceberg_min'], 
                                             temp11['is_iceberg_base']))

In [27]:
#! cp vgg_fcn.ipynb vgg_angle.ipynb
temp11.to_csv('others/vgg_mob_10fold_10.csv',index=False)
# temp11.to_csv('training_set_result/vgg_mob_10fold_10.csv',index=False)